# Experiments for Graph Matching
This notebook contains the experiments to compare the graph matching results via embeddings produced by a GNN to graph matching results via embeddings that are represented by temporal centralities. 

In [2]:
import numpy as np

In [3]:
temporal_betweenness_vectors_sources = np.load('../data/temporal_betweenness_vectors_overtime.npy')
temporal_closeness_vectors_sources = np.load('../data/temporal_closeness_vectors_overtime.npy')
temporal_degree_vectors_sources = np.load('../data/temporal_degree_vectors_overtime.npy')

In [4]:
temporal_betweenness_vectors_targets = np.load('artificial-graph-generation/generated/temporal_betweennesses_unique.npy')
temporal_closeness_vectors_targets = np.load('artificial-graph-generation/generated/temporal_closenesses_unique.npy')
temporal_degree_vectors_targets = np.load('artificial-graph-generation/generated/temporal_degrees_unique.npy')

In [5]:
source_encodings = np.load("../data/anomaly_source_embeddings_17_dims_30_epochs.npy")
target_encodings = np.load("../data/anomaly_target_embeddings_17_dims_30_epochs.npy")

In [6]:
num_sources = len(temporal_betweenness_vectors_sources)
assert len(temporal_closeness_vectors_sources) == len(temporal_degree_vectors_sources) and len(temporal_closeness_vectors_sources) == num_sources
num_targets = len(temporal_betweenness_vectors_targets)
assert len(temporal_closeness_vectors_targets) == len(temporal_degree_vectors_targets) and len(temporal_closeness_vectors_targets) == num_targets


## Comparison of GNN to temporal betweenness

In [7]:
from scipy.spatial import distance
from tqdm import tqdm
from sklearn.metrics import dcg_score
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm

In [8]:
# Temporal closeness
temporal_closeness_distances = []
temporal_closeness_rankings = []
for i in tqdm(range(num_sources)):
    temporal_closeness_distances.append([])
    for j in range(num_targets):
        temporal_closeness_distances[-1].append(distance.euclidean(temporal_betweenness_vectors_sources[i], temporal_betweenness_vectors_targets[j]))
    temporal_closeness_rankings.append(np.argsort(temporal_closeness_distances[-1]))

# Embeddings
embedding_distances = []
embedding_rankings = []
for i in tqdm(range(num_sources)):
    embedding_distances.append([])
    for j in range(num_targets):
        embedding_distances[-1].append(distance.euclidean(source_encodings[i], target_encodings[j]))
        embedding_rankings.append(np.argsort(embedding_distances[-1]))

scores = []
for i in tqdm(range(num_sources)):
    scores.append(dcg_score(temporal_closeness_rankings[i], embedding_rankings[i]))

  0%|          | 2/18312 [05:56<907:45:07, 178.48s/it]

: 

: 

In [ ]:
# TODO: do the graph matching ranking (as Til implemented), compare rankings of GNN and TCMs (comparison of top-k via DCG where the TCM is the truth?)
matching_scores_GNN_betweenness = []
euc_distances_betweenness = {}

for source_idx in tqdm(range(5)):
    source_temporal_closeness = temporal_closeness_vectors_sources[source_idx]
    #source_GNN = ...
    euc_distances_betweenness[source_idx] = []

    for target_idx in range(5):
        target_temporal_betweenness = temporal_betweenness_vectors_targets[target_idx]
        NearestNeighbors(n_neighbors=)
        euc_distances_betweenness[source_idx].append(distance.euclidean(temporal_betweenness_vectors_sources, target_temporal_betweenness))
        
        

    #source_GNN = GNN_vectors_sources[source_idx]
    # TODO: do the graph matching for GNN

    # TODO: compare both matchings and compute a score
    #matching_scores_GNN_betweenness.append(score)
# TODO: somehow aggregate the scores and generate a total score (average? histograms?)



In [25]:
# TODO: do the graph matching ranking (as Til implemented), compare rankings of GNN and TCMs (comparison of top-k via DCG where the TCM is the truth?)
matching_scores_GNN_closeness = []
euc_distances_closeness = {}




for source_idx in tqdm(range(5)):
    source_temporal_closeness = temporal_closeness_vectors_sources[source_idx]
    #source_GNN = ...
    euc_distances_closeness[source_idx] = []

    for target_idx in range(5):
        target_temporal_closeness = temporal_closeness_vectors_targets[target_idx]
        euc_distances_closeness[source_idx].append(distance.euclidean(source_temporal_closeness, target_temporal_closeness))
    
    # TODO: do the graph matching for closeness
    #rank vectors with index ranking based on euclidean distance
    ranking_tcms = ...


    #source_GNN = GNN_vectors_sources[source_idx]
    # TODO: do the graph matching for GNN
    ranking_gnn = ...

    # TODO: compare both matchings and compute a score
    matching_score = dcg_score(y_true=ranking_tcms, y_score=ranking_gnn)
    matching_scores_GNN_closeness.append(matching_score)
# TODO: somehow aggregate the scores and generate a total score (average? histograms?)




100%|██████████| 5/5 [00:00<00:00, 3342.07it/s]


In [26]:
distance.euclidean(temporal_closeness_vectors_sources[0], temporal_closeness_vectors_targets[0])

1.0019512213675874

In [27]:
temporal_closeness_vectors_sources[:5]

array([[0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        1. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        1. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        1.5, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        1. , 1. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 1.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        1. , 0. , 0. , 0. ]])

In [28]:
temporal_closeness_vectors_targets[:5]

array([[0.        , 0.        , 0.        , 0.0625    , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.125     , 0.        ,
        0.        , 0.        , 0.06666667, 0.06666667, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.06666667],
       [0.        , 0.        , 0.        , 0.1875    , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.13809524],
       [0.07692308, 0.        , 0.        , 0.25      , 0.        ,
        0.07692308, 0.        , 0.        , 0.07692308, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.2047619 ],
       [0.16025641, 0.        , 0.        , 0.3125  

In [29]:
euc_distances_closeness

{0: [1.0019512213675874,
  1.0143758343598952,
  1.026755348067192,
  1.0593294657148287,
  1.0994754003353842],
 1: [1.4155939566132656,
  1.4244150846341572,
  1.4332573198084775,
  1.4029728304872873,
  1.4335271194143833],
 2: [1.5013015186830392,
  1.5096219173466359,
  1.5179678997872716,
  1.5401879485737007,
  1.5680708389427607],
 3: [1.733178077982756,
  1.740390281900394,
  1.7476345569896987,
  1.7228850115679544,
  1.747855829894588],
 4: [1.8038587112077265,
  1.8107894226920294,
  1.8177531583756306,
  1.7724022359956653,
  1.7966849821750674]}

In [ ]:
# TODO: do the same for closeness and degree

In [ ]:
# TODO: compare scores for closeness, betweenness and degree and find out something meaningful (regarding the use cases)